In [1]:
import gradio as gr
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
from PIL import Image
import json
import os
import copy
import numpy as np
from util import *
import cv2

face_detector = pipeline(Tasks.face_detection, model='gaosheng/face_detect')
# face_recognizer = pipeline(Tasks.face_recognition, model='damo/cv_ir101_facerecognition_cfglint')
face_recognizer = pipeline(Tasks.face_recognition, model='iic/cv_ir101_facerecognition_cfglint')
emotion_recognizer = pipeline(Tasks.facial_expression_recognition, 'damo/cv_vgg19_facial-expression-recognition_fer')
portrait_matting = pipeline(Tasks.portrait_matting, model='damo/cv_unet_image-matting')
face_bank = load_face_bank('face_bank/', face_recognizer)
name_box_map = {}
detected_image = None


def inference(img: Image, draw_detect_enabled, detect_threshold, sim_threshold) -> json:
    img = resize_img(img)
    img = img.convert('RGB')
    
    global detected_image
    detected_image = copy.deepcopy(img)

    detection_result = face_detector(img)
    boxes = np.array(detection_result[OutputKeys.BOXES])
    scores = np.array(detection_result[OutputKeys.SCORES])
    faces = []

    for i in range(len(boxes)):
        score = scores[i]
        if score < detect_threshold:
            continue
        box = boxes[i]
        face_embedding = get_face_embedding(img, box, face_recognizer)
        name, sim = get_name_sim(face_embedding, face_bank)
        if name is None:
            continue
        if sim < sim_threshold:
            faces.append({'box': box, 'name': '未知', 'sim': sim})
        else:
            faces.append({'box': box, 'name': name, 'sim': sim})
            real_name = name[2:] # 去掉前2位学号
            name_box_map[real_name] = box
    rows = get_rows(faces)
    row_names = get_row_names(faces, rows)
    draw_name(img, row_names)
    if draw_detect_enabled:
        draw_faces(img, faces, emotion_recognizer)
    return img, get_row_names_text(row_names)

def search_face_cutouts(name):
    if name not in name_box_map:
        return "404.jpg"

    # 适当扩大边框范围，保证覆盖人脸但是又不会显得边框过大
    box = name_box_map[name]
    box[0] = box[0] - 5
    box[2] = box[2] + 2
    box[1] = box[1] - 2
    box[3] = box[3] + 2

    face_img = get_face_img(detected_image, box)
    result = portrait_matting(face_img)
    face_cutouts = result[OutputKeys.OUTPUT_IMG]
    # 要先写成本地图片才能保存颜色(原始的npy array会导致丢部分颜色信息)
    cv2.imwrite('temp.png', face_cutouts)
    # 抠图之后的图像太小，需要等比放大一些
    face_cutouts = Image.open('temp.png')
    original_width, original_height = face_cutouts.size
    new_width = original_width * 5
    new_height = original_height * 5
    resized_face_cutouts = face_cutouts.resize((new_width, new_height), Image.LANCZOS)
    return resized_face_cutouts

examples = ['example.jpg']

with gr.Blocks() as demo:
    with gr.Row():
        draw_detect_enabled = gr.Checkbox(label="是否画框", value=True)
        detect_threshold = gr.Slider(label="检测阈值", minimum=0, maximum=1, value=0.3)
        sim_threshold = gr.Slider(label="识别阈值", minimum=0, maximum=1, value=0.3)
    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="pil", height=350)
            submit = gr.Button("提交")
        with gr.Column():
            img_output = gr.Image(type="pil")
            name_output = gr.Text(label="人名")
    with gr.Row():
        with gr.Column():
            name_input = gr.Text(label="输入人名搜索头像")
            submit2 = gr.Button("提交")
        with gr.Column():
            face_cutouts = gr.Image(type="pil")
    submit.click(
        fn=inference,
        inputs=[img_input, draw_detect_enabled, detect_threshold, sim_threshold],
        outputs=[img_output, name_output])
    submit2.click(
        fn=search_face_cutouts,
        inputs=[name_input],
        outputs=[face_cutouts])
    gr.Examples(examples, inputs=[img_input])

demo.launch(share=True)

2024-12-09 21:09:08.290998: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 21:09:08.330886: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 21:09:09.177614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-12-09 21:09:11,444 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect
2024-12-09 21:09:11,445 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect.
2024-12-09 21:09:11,449 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect
/usr/local/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumen

load checkpoint from local path: /mnt/workspace/.cache/modelscope/hub/gaosheng/face_detect/pytorch_model.pt


2024-12-09 21:09:20,659 - modelscope - INFO - load model done


2024-12-09 21:09:21,488 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2024-12-09 21:09:21,808 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint
2024-12-09 21:09:21,809 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint.
2024-12-09 21:09:21,819 - modelscope - WARNING - No preprocessor field found in cfg.
2024-12-09 21:09:21,819 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-12-09 21:09:21,820 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/hub/iic/cv_ir101_facerecognition_cfglint'}. trying to build by task and model information.
2024-12-09 21:09:21,820 - modelscope - WARNING - Find task: face-recognition, model type: None. Insufficient information 

...

2024-12-09 21:09:27.877018: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-09 21:09:27.878529: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-09 21:09:27.878682: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Instructions for updating:
Use tf.gfile.GFile.


2024-12-09 21:09:28,150 - modelscope - INFO - load model done


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://9cbda49a0fa2b7552b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet/core/anchor/anchor_generator.py:333: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/usr/local/lib/python3.10/site-packages/mmdet/core/anchor/anchor_generator.py:369: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  warnings.warn(
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet